# Cat vs Dog Classification using Deep Neural Networks

This notebook explores multiple deep learning models to classify images of cats and dogs using the `cats_vs_dogs` dataset from TensorFlow Datasets. It compares different DNN architectures to evaluate their performance and identify the most effective model.

In [ ]:
# 1. Import the required libraries
# =================================

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

## 2. Data Acquisition & Pre-processing

In [ ]:
# 2.1 Load the dataset
# --------------------
(train_ds, val_ds, test_ds), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

In [ ]:
# 2.2 Visualize the data
# ----------------------
print("--- Sample Images from the Training Dataset ---")
plt.figure(figsize=(10, 5))
for i, (image, label) in enumerate(train_ds.take(10)):
    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(image)
    plt.title('Cat' if label == 0 else 'Dog')
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# 2.3 Pre-process the data
# ------------------------
IMG_SIZE = 150
BATCH_SIZE = 32

def preprocess(image, label):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

train_ds = train_ds.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print(f"\nData pre-processing complete. Images resized to {IMG_SIZE}x{IMG_SIZE} and batched.")

## 3. Code for the Deep Neural Network Models

In [ ]:
# Model 1: Basic CNN
model1 = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
], name="Basic_CNN")

# Model 2: Deeper CNN
model2 = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
], name="Deeper_CNN")

# Model 3: CNN with Dropout
model3 = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
], name="CNN_with_Dropout")

# Model 4: CNN with Batch Normalization
model4 = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name="CNN_with_Batch_Norm")

# Model 5: CNN with L2 Regularization
model5 = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dense(1, activation='sigmoid')
], name="CNN_with_L2_Regularization")

models = [model1, model2, model3, model4, model5]
for model in models:
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print(f"--- {model.name} Summary ---")
    model.summary()
    print("\n")

## 4. Train and Evaluate the Models

In [ ]:
histories = []
epochs = 10

for model in models:
    print(f"\n--- Training {model.name} ---")
    history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)
    histories.append({'name': model.name, 'history': history})

In [ ]:
def plot_metric(histories, metric, title):
    plt.figure(figsize=(12, 8))
    for item in histories:
        name = item['name']
        history = item['history']
        val_metric = f'val_{metric}'
        plt.plot(history.history[metric], label=f'{name} Train {metric.capitalize()}', lw=2)
        plt.plot(history.history[val_metric], label=f'{name} Val {metric.capitalize()}', linestyle='--')
    plt.title(title, fontsize=16)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel(metric.capitalize(), fontsize=12)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

plot_metric(histories, 'accuracy', 'Model Comparison: Accuracy')
plot_metric(histories, 'loss', 'Model Comparison: Loss')

## 5. Test the Best Model

In [ ]:
best_model = model4

print(f"\n--- Evaluating Best Model ({best_model.name}) on Test Data ---")
loss, accuracy = best_model.evaluate(test_ds)
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Test Loss: {loss:.4f}')

y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred_probs = best_model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype("int32").flatten()

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=['Cat', 'Dog']))

print("\n--- Confusion Matrix ---")
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Cat', 'Dog'])
disp.plot(cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix for {best_model.name}')
plt.show()

## 6. Conclusion

This notebook demonstrated building, training, and evaluating several deep learning models for image classification.
By comparing different architectures, we selected the one that performs best on our specific task.
Model 4, with Batch Normalization, appears to be a good choice due to its stable training and high accuracy.
Further improvements could be made by tuning hyperparameters, using more advanced architectures (e.g., adding Convolutional layers),
or employing data augmentation techniques.
